In [ ]:
from google.colab import drive

drive.mount('/content/drive')
root_dir = "/content/drive/My Drive/"
base_dir = root_dir + 'Tesi/'

In [ ]:
import pandas as pd 

df = pd.read_csv(base_dir + 'downstream/ccoha2_preprocessed.csv')

In [ ]:
!pip install git+https://github.com/boudinfl/pke.git

In [ ]:
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import pke

In [ ]:
import spacy
!python -m spacy download en
nlp = spacy.load("en")

In [ ]:
from itertools import chain
from collections import Counter

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
import re

In [ ]:
df.head(10)

TOPICRANK

In [ ]:
# select the longest sequences of nouns and adjectives, that do
# not contain punctuation marks or stopwords as candidates.
pos = {'NOUN', 'PROPN', 'ADJ'}
stoplist = list(string.punctuation)
stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
stoplist += stopwords.words('english')

In [ ]:
def extract_keyphrases(caption, n):
    extractor = pke.unsupervised.TopicRank()
    extractor.load_document(caption)
    extractor.candidate_selection(pos = pos, stoplist=stoplist)
    extractor.candidate_weighting(threshold=0.74, method='average')
    keyphrases = extractor.get_n_best(n=n, stemming=False)
    return(keyphrases)

In [ ]:
df['keywords'] = df.progress_apply(lambda row: (extract_keyphrases(row['text'], 10)),axis=1)

In [ ]:
df.head(20)

In [ ]:
# IF MWE ALREADY EXTRACTED

import pandas as pd 

df = pd.read_csv(base_dir + 'downstream/checkpoints/checkpointCCOHA2_topicrank.csv', converters={'keywords': eval})

In [ ]:
df

In [ ]:
keywords_without_unigrams = []

In [ ]:
for i in range(len(df)):
  for kw in df['keywords'][i]:
    if(' ' in kw[0]):
        keywords_without_unigrams.append(kw[0])
  df['keywords'][i]=keywords_without_unigrams
  keywords_without_unigrams = []

In [ ]:
df['keywords']

MWE THRESHOLD

In [ ]:
mwe_counter = Counter(chain.from_iterable(df['keywords'])) 

In [ ]:
df['keywords'] = [([j for j in i if mwe_counter[j] >= 5]) for i in df['keywords']]

SUBSTITUTION

In [ ]:
for i in range(len(df)):
  df['text'][i] = re.sub('('+'|'.join('\\b'+re.escape(g)+'\\b' for g in df['keywords'][i])+')', lambda m: m.group(0).replace(' ', '_'), df['text'][i], flags=re.I)

In [ ]:
df

In [ ]:
compression_opts = dict(method='zip', archive_name='newCCOHA2_topicrank.csv')

df['text'].to_csv('newCCOHA2_topicrank.zip', index=False, compression=compression_opts)